<a href="https://colab.research.google.com/github/yijunhou/CS229_Project/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import csv
import math
from collections import Counter
import pandas as pd
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
composer_lookup = {"Beethoven":0,"Schubert":1,"Brahms":2,"Mozart":3,"Bach":4,"Dvorak":5,"Cambini":6,"Faure":7,"Ravel":8,"Haydn":9}
sr = 44100
fs = 44100
chunk_size, chunk_seconds = 10, 10

In [ ]:
train_data = np.load('/content/gdrive/My Drive/CS229/project/Data/musicnet.npz', encoding='bytes', allow_pickle=True)
y_train_path = '/content/gdrive/My Drive/CS229/project/Data/ytrnew_' + str(sr) + '_' + str(chunk_seconds) + '.npy'
x_train_path = '/content/gdrive/My Drive/CS229/project/Data/xtrnew_' + str(sr) + '_' + str(chunk_seconds) + '.npy'
y_test_path = '/content/gdrive/My Drive/CS229/project/Data/ytenew_' + str(sr) + '_' + str(chunk_seconds) + '.npy'
x_test_path = '/content/gdrive/My Drive/CS229/project/Data/xtenew_' + str(sr) + '_' + str(chunk_seconds) + '.npy'

In [ ]:
def generate_composer_map():
  composer_map = {}
  with open('/content/gdrive/My Drive/CS229/project/Data/musicnet_metadata.csv', 'r' ) as metadata_file:
    reader = csv.DictReader(metadata_file)
    for line in reader:
      composer_map[line['id']] = line['composer']
  return composer_map

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/CS229/project/Data/data_file.csv')

In [ ]:
x_train = np.load(x_train_path)
y_train = np.load(y_train_path)
x_rest = np.load(x_test_path)
y_rest = np.load(y_test_path)

x_validation, x_test, y_validation, y_test = train_test_split(x_rest, y_rest, test_size = 0.5, 
                                                   random_state = 42, stratify = y_rest)

print(x_train.shape, x_validation.shape, x_test.shape)

(4000, 40, 1000, 1) (500, 40, 1000, 1) (501, 40, 1000, 1)


In [ ]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_validation = x_validation.reshape(x_validation.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
print(x_train.shape, x_validation.shape, x_test.shape)

(4000, 40000) (500, 40000) (501, 40000)


In [ ]:
y_train = [np.where(r==1)[0][0] for r in y_train]
y_validation = [np.where(r==1)[0][0] for r in y_validation]
y_test = [np.where(r==1)[0][0] for r in y_test]

print(len(y_train), len(y_validation), len(y_test))

4000 500 501


## **Random Forest**

In [ ]:
model_rf = RandomForestClassifier(random_state=0, max_depth = 10).fit(x_train, y_train)

In [ ]:
print(model_rf.score(x_train, y_train))
print(Counter(y_train - model_rf.predict(x_train)))
# y_predict_30 = model_rf.predict(x_test_30)
print(model_rf.score(x_test, y_test))
print(model_rf.score(x_validation, y_validation))

0.70275
Counter({0: 2811, 1: 366, 4: 303, 3: 264, 2: 114, 5: 63, 8: 61, 9: 18})
0.48902195608782434
0.496


# **SVM**

In [ ]:
model_svm = svm.SVC(random_state=42).fit(x_train,y_train)

In [ ]:
print(model_svm.score(x_train, y_train))
print(Counter(y_train - model_svm.predict(x_train)))
# y_predict_30 = svm_30.predict(x_test_30)
print(model_svm.score(x_validation, y_validation))
print(model_svm.score(x_test, y_test))

0.51575
Counter({0: 2063, 2: 542, 1: 491, 3: 433, 4: 324, 5: 63, 8: 60, 9: 18, 6: 4, -2: 2})
0.5
0.5049900199600799
